In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model

import matplotlib.pyplot as plt

In [3]:
generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2)))
generator.add(BatchNormalization())
generator.add(Reshape((7,7,128)))

In [4]:
generator.add(UpSampling2D())
generator.add(Conv2D(64, kernel_size=5, padding='same'))
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D())
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh'))

In [5]:
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding="same"))
discriminator.add(Activation(LeakyReLU(0.2)))

In [6]:
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable= False

In [7]:
ginput = Input(shape=(100,))

In [8]:
dis_output = discriminator(generator(ginput))
gan = Model(ginput, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')
gan.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
sequential (Sequential)      (None, 28, 28, 1)         865281    
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 212865    
Total params: 1,078,146
Trainable params: 852,609
Non-trainable params: 225,537
_________________________________________________________________


In [ ]:
def gan_train(epoch, batch_size, saving_interval):
  (X_train, _), (_, _) = mnist.load_data() 
  X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
  X_train = (X_train - 127.5) / 127.5  

  true = np.ones((batch_size, 1))
  fake = np.zeros((batch_size, 1))

  for i in range(epoch):
    
          idx = np.random.randint(0, X_train.shape[0], batch_size)
          imgs = X_train[idx]
          d_loss_real = discriminator.train_on_batch(imgs, true)

        
          noise = np.random.normal(0, 1, (batch_size, 100))
          gen_imgs = generator.predict(noise)
          d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)

          
          d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
          g_loss = gan.train_on_batch(noise, true)

          print('epoch:%d' % i, ' d_loss:%.4f' % d_loss, ' g_loss:%.4f' % g_loss)

          if i % saving_interval == 0:
         
              noise = np.random.normal(0, 1, (25, 100))
              gen_imgs = generator.predict(noise)

       
              gen_imgs = 0.5 * gen_imgs + 0.5

              fig, axs = plt.subplots(5, 5)
              count = 0
              for j in range(5):
                  for k in range(5):
                      axs[j, k].imshow(gen_imgs[count, :, :, 0], cmap='gray')
                      axs[j, k].axis('off')
                      count += 1
              fig.savefig("gan_mnist_%d.png" % i) #fig.savefig(파일명), 책은 error뜸. "gan_images/gan_mnist_%d.png"를 "gan_mnist_%d.png"로만 수정.

gan_train(4001, 32, 200)  

11493376/11490434 [==============================] - 0s 0us/step
epoch:0  d_loss:0.7247  g_loss:0.3763
epoch:1  d_loss:0.5355  g_loss:0.1897
epoch:2  d_loss:0.5080  g_loss:0.1265
epoch:3  d_loss:0.4768  g_loss:0.1506
epoch:4  d_loss:0.4760  g_loss:0.2673
epoch:5  d_loss:0.4715  g_loss:0.4434
epoch:6  d_loss:0.4762  g_loss:0.5961
epoch:7  d_loss:0.5675  g_loss:0.6295
epoch:8  d_loss:0.6100  g_loss:0.5462
epoch:9  d_loss:0.7473  g_loss:0.3832
epoch:10  d_loss:0.6072  g_loss:0.2723
epoch:11  d_loss:0.5309  g_loss:0.2410
epoch:12  d_loss:0.4602  g_loss:0.2168
epoch:13  d_loss:0.4166  g_loss:0.1952
epoch:14  d_loss:0.4028  g_loss:0.2129
epoch:15  d_loss:0.3996  g_loss:0.1840
epoch:16  d_loss:0.3838  g_loss:0.2584
epoch:17  d_loss:0.3763  g_loss:0.3721
epoch:18  d_loss:0.3485  g_loss:0.4798
epoch:19  d_loss:0.3842  g_loss:0.3688
epoch:20  d_loss:0.4136  g_loss:0.3990
epoch:21  d_loss:0.4569  g_loss:0.4449
epoch:22  d_loss:0.4946  g_loss:0.5874
epoch:23  d_loss:0.6344  g_loss:0.4399
epoch:24 

epoch:206  d_loss:0.3596  g_loss:2.1121
epoch:207  d_loss:0.3900  g_loss:2.5095
epoch:208  d_loss:0.3703  g_loss:2.2597
epoch:209  d_loss:0.4640  g_loss:2.3269
epoch:210  d_loss:0.4219  g_loss:2.1264
epoch:211  d_loss:0.4666  g_loss:1.9164
epoch:212  d_loss:0.4408  g_loss:1.7387
epoch:213  d_loss:0.6287  g_loss:1.5672
epoch:214  d_loss:0.3921  g_loss:2.2738
epoch:215  d_loss:0.4526  g_loss:2.4616
epoch:216  d_loss:0.4685  g_loss:2.2338
epoch:217  d_loss:0.5827  g_loss:1.8416
epoch:218  d_loss:0.8260  g_loss:1.3177
epoch:219  d_loss:0.5289  g_loss:1.5236
epoch:220  d_loss:0.5693  g_loss:1.9997
epoch:221  d_loss:0.6133  g_loss:2.1627
epoch:222  d_loss:0.4892  g_loss:2.4187
epoch:223  d_loss:0.5926  g_loss:1.8481
epoch:224  d_loss:0.5714  g_loss:1.5798
epoch:225  d_loss:0.5283  g_loss:1.3505
epoch:226  d_loss:0.5110  g_loss:1.6577
epoch:227  d_loss:0.4853  g_loss:1.7646
epoch:228  d_loss:0.4529  g_loss:2.1312
epoch:229  d_loss:0.4158  g_loss:2.2044
epoch:230  d_loss:0.4686  g_loss:2.4518


epoch:411  d_loss:0.3330  g_loss:2.2221
epoch:412  d_loss:0.2229  g_loss:2.5212
epoch:413  d_loss:0.3216  g_loss:2.3203
epoch:414  d_loss:0.2517  g_loss:2.5776
epoch:415  d_loss:0.2692  g_loss:2.3947
epoch:416  d_loss:0.5009  g_loss:1.8278
epoch:417  d_loss:0.3418  g_loss:1.9669
epoch:418  d_loss:0.4222  g_loss:2.0760
epoch:419  d_loss:0.2160  g_loss:2.9108
epoch:420  d_loss:0.2902  g_loss:2.8070
epoch:421  d_loss:0.3065  g_loss:3.3308
epoch:422  d_loss:0.3692  g_loss:2.3043
epoch:423  d_loss:0.3997  g_loss:1.8653
epoch:424  d_loss:0.2932  g_loss:1.9249
epoch:425  d_loss:0.2041  g_loss:2.5963
epoch:426  d_loss:0.3544  g_loss:2.8309
epoch:427  d_loss:0.2618  g_loss:3.0895
epoch:428  d_loss:0.3098  g_loss:2.4036
epoch:429  d_loss:0.3202  g_loss:2.1559
epoch:430  d_loss:0.2821  g_loss:2.1768
epoch:431  d_loss:0.2072  g_loss:2.6998
epoch:432  d_loss:0.2512  g_loss:3.0139
epoch:433  d_loss:0.2490  g_loss:2.6224
epoch:434  d_loss:0.1138  g_loss:2.7855
epoch:435  d_loss:0.2088  g_loss:2.4404


epoch:616  d_loss:0.9379  g_loss:0.8673
epoch:617  d_loss:0.8982  g_loss:0.6455
epoch:618  d_loss:0.8773  g_loss:0.6486
epoch:619  d_loss:0.7274  g_loss:0.8225
epoch:620  d_loss:0.5181  g_loss:1.1088
epoch:621  d_loss:0.5245  g_loss:1.3696
epoch:622  d_loss:0.4436  g_loss:1.5524
epoch:623  d_loss:0.4340  g_loss:1.7728
epoch:624  d_loss:0.2968  g_loss:2.1286
epoch:625  d_loss:0.3465  g_loss:2.0235
epoch:626  d_loss:0.2696  g_loss:2.2758
epoch:627  d_loss:0.2220  g_loss:2.4871
epoch:628  d_loss:0.3197  g_loss:2.2039
epoch:629  d_loss:0.3966  g_loss:2.6013
epoch:630  d_loss:0.2485  g_loss:2.3303
epoch:631  d_loss:0.3837  g_loss:2.2817
epoch:632  d_loss:0.4305  g_loss:2.3854
epoch:633  d_loss:0.4143  g_loss:1.8084
epoch:634  d_loss:0.6106  g_loss:1.8825
epoch:635  d_loss:0.4987  g_loss:1.8067
epoch:636  d_loss:0.4456  g_loss:1.4554
epoch:637  d_loss:0.5656  g_loss:1.5625
epoch:638  d_loss:0.5742  g_loss:1.4868
epoch:639  d_loss:0.5230  g_loss:1.4239
epoch:640  d_loss:0.4919  g_loss:1.5315


epoch:821  d_loss:0.4619  g_loss:1.9457
epoch:822  d_loss:0.3991  g_loss:2.2619
epoch:823  d_loss:0.5580  g_loss:1.6443
epoch:824  d_loss:0.6687  g_loss:1.7455
epoch:825  d_loss:0.5968  g_loss:1.1934
epoch:826  d_loss:0.6357  g_loss:1.3039
epoch:827  d_loss:0.5112  g_loss:1.3083
epoch:828  d_loss:0.5362  g_loss:1.5916
epoch:829  d_loss:0.3878  g_loss:2.2440
epoch:830  d_loss:0.5191  g_loss:2.0194
epoch:831  d_loss:0.6521  g_loss:1.6797
epoch:832  d_loss:0.6514  g_loss:1.2025
epoch:833  d_loss:0.6712  g_loss:1.2192
epoch:834  d_loss:0.5194  g_loss:1.5421
epoch:835  d_loss:0.4346  g_loss:1.7667
epoch:836  d_loss:0.3383  g_loss:1.7667
epoch:837  d_loss:0.4642  g_loss:1.9006
epoch:838  d_loss:0.3632  g_loss:2.4537
epoch:839  d_loss:0.4870  g_loss:1.9737
epoch:840  d_loss:0.3537  g_loss:1.9712
epoch:841  d_loss:0.3470  g_loss:2.0173
epoch:842  d_loss:0.3445  g_loss:2.1442
epoch:843  d_loss:0.2882  g_loss:2.2897
epoch:844  d_loss:0.3105  g_loss:2.2724
epoch:845  d_loss:0.2352  g_loss:2.2552


epoch:1030  d_loss:0.3757  g_loss:1.8525
epoch:1031  d_loss:0.4403  g_loss:1.7698
epoch:1032  d_loss:0.2636  g_loss:2.0446
epoch:1033  d_loss:0.3133  g_loss:2.1274
epoch:1034  d_loss:0.3550  g_loss:1.8140
epoch:1035  d_loss:0.3995  g_loss:1.8428
epoch:1036  d_loss:0.3566  g_loss:1.8554
epoch:1037  d_loss:0.2977  g_loss:1.7457
epoch:1038  d_loss:0.2642  g_loss:2.1100
epoch:1039  d_loss:0.3169  g_loss:2.5365
epoch:1040  d_loss:0.3079  g_loss:1.9890
epoch:1041  d_loss:0.4380  g_loss:1.9471
epoch:1042  d_loss:0.5411  g_loss:1.5850
epoch:1043  d_loss:0.3385  g_loss:1.5693
epoch:1044  d_loss:0.3779  g_loss:1.7295
epoch:1045  d_loss:0.4003  g_loss:2.1734
epoch:1046  d_loss:0.3144  g_loss:2.5784
epoch:1047  d_loss:0.3759  g_loss:2.4139
epoch:1048  d_loss:0.3618  g_loss:2.2637
epoch:1049  d_loss:0.2759  g_loss:2.3157
epoch:1050  d_loss:0.3152  g_loss:1.9286
epoch:1051  d_loss:0.4020  g_loss:1.9014
epoch:1052  d_loss:0.2404  g_loss:2.2005
epoch:1053  d_loss:0.3941  g_loss:2.3186
epoch:1054  d_lo

epoch:1231  d_loss:0.4441  g_loss:1.7056
epoch:1232  d_loss:0.5138  g_loss:2.1685
epoch:1233  d_loss:0.6224  g_loss:2.4198
epoch:1234  d_loss:0.7166  g_loss:2.1854
epoch:1235  d_loss:0.4491  g_loss:1.6924
epoch:1236  d_loss:0.4698  g_loss:1.5911
epoch:1237  d_loss:0.4277  g_loss:1.9105
epoch:1238  d_loss:0.3845  g_loss:1.9481
epoch:1239  d_loss:0.3949  g_loss:2.1678
epoch:1240  d_loss:0.4726  g_loss:2.2241
epoch:1241  d_loss:0.3267  g_loss:1.9776
epoch:1242  d_loss:0.4011  g_loss:1.8504
epoch:1243  d_loss:0.3833  g_loss:2.0059
epoch:1244  d_loss:0.2712  g_loss:2.1470
epoch:1245  d_loss:0.2638  g_loss:2.4245
epoch:1246  d_loss:0.2681  g_loss:2.5845
epoch:1247  d_loss:0.2273  g_loss:2.0780
epoch:1248  d_loss:0.5391  g_loss:2.3666
epoch:1249  d_loss:0.3770  g_loss:2.5044
epoch:1250  d_loss:0.4068  g_loss:2.3963
epoch:1251  d_loss:0.4357  g_loss:2.3325
epoch:1252  d_loss:0.5038  g_loss:1.7721
epoch:1253  d_loss:0.3854  g_loss:2.1185
epoch:1254  d_loss:0.2688  g_loss:2.1135
epoch:1255  d_lo